In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import snscrape.modules.twitter as sntwitter

sns.set(rc = {'figure.figsize':(15, 7)})
sns.set_style('white')
pd.set_option('display.max_columns', 500)

In [9]:
tweets = pd.read_csv('./data/cleaned_up_data/final_stock_tweets_summary.csv', parse_dates=['Date'])

tweets.head()

,Date,Stock Name,sentiment avg,label,sentiment pct,Open,High,Low,Close,Adj Close,Volume
0,2015-01-02,AAPL,0.688692,bearish,0.306551,27.847500,27.860001,26.837500,27.332500,24.603210,212818400
1,2015-01-02,AAPL,0.688692,bullish,0.693449,27.847500,27.860001,26.837500,27.332500,24.603210,212818400
2,2015-01-02,AMZN,0.643557,bearish,0.345455,15.629000,15.737500,15.348000,15.426000,15.426000,55664000
3,2015-01-02,AMZN,0.643557,bullish,0.654545,15.629000,15.737500,15.348000,15.426000,15.426000,55664000
4,2015-01-02,CRM,0.781111,bearish,0.222222,59.900002,60.430000,58.509998,59.240002,59.240002,2796400


### Scraping Twitter

In [14]:
tweets['Date'].max()

Timestamp('2022-09-28 00:00:00')

In [263]:
twt_list = []

for ticker in lst_to_scrape:
    for idx, tweet in enumerate(sntwitter.TwitterSearchScraper(
        f'@{ticker} since:2022-09-28 until:2023-03-31').get_items()):
        
        if idx > 10000:
            break

        twt_list.append([tweet.date, tweet.rawContent, ticker])
        
for ticker in tweets2['Stock Name'].unique():
    for idx, tweet in enumerate(sntwitter.TwitterSearchScraper(
        f'@{ticker} since:2020-01-01 until:2021-09-29').get_items()):
        
        if idx > 15000:
            break

        twt_list2.append([tweet.date, tweet.rawContent, ticker])

In [248]:
df = pd.DataFrame(twt_list, columns=['Date', 'Tweet', 'Stock Name']).merge(
    tweets[['Stock Name', 'Company Name']].drop_duplicates(),
    on='Stock Name')

df2 = pd.DataFrame(twt_list2, columns=['Date', 'Tweet', 'Stock Name']).merge(
    tweets[['Stock Name', 'Company Name']].drop_duplicates(),
    on='Stock Name')

In [250]:
df.to_csv('./data/scraped_tweets_2015_2019.csv', index=False)
df2.to_csv('./data/scraped_tweets_2020_2021.csv', index=False)

### Scraping Yahoo Finances

In [20]:
from yahoo_fin.stock_info import get_analysts_info
import yahoo_fin.stock_info as si

In [21]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'

In [27]:
si.get_data(ticker, start_date = '09/28/2022', end_date = '09/29/2023').reset_index()

,index,open,high,low,close,adjclose,volume,ticker
0,2022-09-28,147.639999,150.639999,144.839996,149.839996,149.363785,146691400,AAPL
1,2022-09-29,146.100006,146.720001,140.679993,142.479996,142.027176,128138200,AAPL
2,2022-09-30,141.279999,143.100006,138.000000,138.199997,137.760773,124925300,AAPL
3,2022-10-03,138.210007,143.070007,137.690002,142.449997,141.997269,114311700,AAPL
4,2022-10-04,145.029999,146.220001,144.259995,146.100006,145.635681,87830100,AAPL
...,...,...,...,...,...,...,...,...
101,2023-02-23,150.089996,150.339996,147.240005,149.399994,149.399994,48394200,AAPL
102,2023-02-24,147.110001,147.190002,145.720001,146.710007,146.710007,55469600,AAPL
103,2023-02-27,147.710007,149.169998,147.449997,147.919998,147.919998,44998500,AAPL
104,2023-02-28,147.050003,149.080002,146.830002,147.410004,147.410004,50547000,AAPL


In [28]:
# stocks = pd.read_csv('./data/scraped_stocks/scraped_stock_2015_2022.csv').iloc[:, 1:]

In [61]:
df_list = []

for ticker in tweets['Stock Name'].unique():
    try:
        data = si.get_data(ticker, start_date = '09/28/2022', end_date = '09/29/2023').reset_index()
        data.columns = stocks.columns

        globals()[ticker] = data
        df_list.append(globals()[ticker])
    except:
        data = si.get_data(ticker, start_date = '09/28/2022', end_date = '09/29/2023').reset_index()
        data.columns = stocks.columns

        globals()[ticker] = data
        df_list2.append(globals()[ticker])

In [67]:
df = pd.concat(df_list)
df = pd.concat([df, stocks])
df['Date'] = pd.to_datetime(df.Date)
df.to_csv('./data/cleaned_up_data/scraped_stock_2015_2023.csv', index_label=False)

In [68]:
pd.read_csv('./data/cleaned_up_data/scraped_stock_2015_2023.csv')

,Date,Open,High,Low,Close,Adj Close,Volume,Stock Name
0,2022-09-28,147.639999,150.639999,144.839996,149.839996,149.363785,146691400,AAPL
1,2022-09-29,146.100006,146.720001,140.679993,142.479996,142.027176,128138200,AAPL
2,2022-09-30,141.279999,143.100006,138.000000,138.199997,137.760773,124925300,AAPL
3,2022-10-03,138.210007,143.070007,137.690002,142.449997,141.997284,114311700,AAPL
4,2022-10-04,145.029999,146.220001,144.259995,146.100006,145.635681,87830100,AAPL
...,...,...,...,...,...,...,...,...
45435,2022-09-22,13.730000,14.600000,13.400000,13.740000,13.740000,23489400,XPEV
45436,2022-09-23,13.090000,13.892000,12.860000,13.710000,13.710000,28279600,XPEV
45437,2022-09-26,14.280000,14.830000,14.070000,14.370000,14.370000,27891300,XPEV
45438,2022-09-27,14.580000,14.800000,13.580000,13.710000,13.710000,21160800,XPEV
